In [1]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import sys
import pandas as pd
import numpy as np
import csv
import random
import re
from sklearn.decomposition import TruncatedSVD
from torch.nn.utils.rnn import pad_sequence#用来填充序列
import time
from prefetch_generator import BackgroundGenerator
from torch.utils.tensorboard import SummaryWriter
from pywick.optimizers.nadam import Nadam#使用pywick包里的nadam优化器

In [2]:
with open('/home/jsy/data/cidlist_complete.csv') as f:
    reader = csv.reader(f)
    cidlist = [row[1] for row in reader]#得到cid对应表
cidlist = list(map(float,cidlist))#把各个元素字符串类型转成浮点数类型
class DataLoaderX(DataLoader):
    def __iter__(self):
        return BackgroundGenerator(super().__iter__())
class BisaiDataset(Dataset):#数据预处理器
    def __init__(self,filepath):
        self.lablelist = pd.read_csv('/home/jsy/data/lablelist.csv',index_col = 0)#比赛id及其对应赛果的列表
        self.filelist0 = [i+'/'+k for i,j,k in os.walk(filepath) for k in k]#得到所有csv文件的路径列表
        self.filelist = [data_path for data_path in self.filelist0 if int(re.findall(r'/(\d*?).csv',data_path)[0]) in  self.lablelist.index]#只保留有赛果的文件路径
        self.lables = {'win':0,'draw':1,'lose':2}#分类问题要从0开始编号,而且要对应好了表中的顺序编，
        if
    
    def __getitem__(self, index):
        #todo
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        #这里需要注意的是，第一步：read one data，是一个dat
        data_path = self.filelist[index]
        bisai_id = int(re.findall(r'/(\d*?).csv',data_path)[0])
        # 2. Preprocess the data (e.g. torchvision.Transform).
        data = self.csv2frame(data_path)
        # 3. Return a data pair (e.g. image and label).
        lable = self.lablelist.loc[bisai_id].result
        lable = self.lables[lable]
        return data,lable      
       
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return len(self.filelist)

    def csv2frame(self,filepath):#给出单场比赛的csv文件路径，并转化成帧列表和对应变帧时间列表，以及比赛结果
        data = pd.read_csv(filepath)#读取文件
        data = data.drop(columns=['league','zhudui','kedui','companyname'])#去除非数字的列
        frametimelist=data.frametime.value_counts().sort_index(ascending=False).index#将frametime的值读取成列表
        framelist =list()#framelist为一个空列表,长度与frametimelist相同,一定要规定好具体形状和float类型，否则dataloader无法读取
        '''
        此处两个循环算法太慢，用pandas更慢，完全抛弃pandas后，数据处理速度从109秒降到了10秒，降到10秒后cpu利用率20%，再往上提也提不上去了，可能需要C++或C来写了
        '''
        new_data = np.array(data)
        lables = new_data[:,0]
        if len(frametimelist)>250:
            frametimelist = [frametimelist[0]]+random.sample(list(frametimelist)[1:-1],248)+[frametimelist[-1]]#如果长度大于500,保留头尾，并在中间随机抽取498个，共计500个
            frametimelist.sort(reverse=True)#并降序排列
        for i in frametimelist:
            state = new_data[lables==i]#从第一次变盘开始得到当次转移
            #state = np.array(state)#不必转成numpy多维数组，因为已经是了
            state = np.delete(state,(0,1), axis=-1)#去掉frametime和cid
            #在填充成矩阵之前需要知道所有数据中到底有多少个cid
            framelist.append(state)
        frametimelist = np.array(frametimelist)
        vectensor = self.mrx2vec(framelist)
        len_frame = vectensor.shape[0]
        if len_frame<250:
            vectensor = np.concatenate((np.zeros((250-len_frame,10),dtype=np.float64),vectensor),axis=0)#如果不足500，则在前面用0填充
        vectensor = torch.from_numpy(vectensor)
        return vectensor#传出一个帧列表,也可以把frametimelist一并传出来，此处暂不考虑位置参数的问题
    
    def tsvd(self,frame):
        tsvd = TruncatedSVD(1)
        if frame.shape[0] != 1:
            newframe = tsvd.fit_transform(np.transpose(frame))#降维成（1,10）的矩阵
        else:
            return frame.reshape((10,1))#第一行需要reshape一下
        return newframe
    
    def mrx2vec(self,flist):#把截断奇异值的方法把矩阵变成向量(matrix2vec/img2vec)，传入：len(frametimelist)*(306*10),传出：len(frametimelist)*10
        vectensor = np.array(list(map(self.tsvd,flist))).squeeze(2)
        #veclist = veclist.transpose()
        #vectensor = torch.from_numpy(veclist)#转成张量
        return vectensor#传出一个形状为(1,序列长度,10)的张量，因为后面传入模型之前，还需要做一下pad_sequence(0维是batch_size维)

 

class Lstm(nn.Module):#把CNN的结果输入LSTM里
    def __init__(self):
        super().__init__()
        self.encoder = nn.LSTM(input_size=10, 
                                hidden_size=250,#选择对帧进行保留首尾的均匀截断采样
                                num_layers=1,#暂时就只有一层
                                bidirectional=True)
        self.decoder = nn.Sequential(
            nn.Linear(1000, 250),#把LSTM的输出
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(250, 3)
        )

    def forward(self,inputs):
        output, _= self.encoder(inputs.permute(1,0,2))#inputs需要转置一下再输入lstm层，因为pytorch要求第0维为长度，第二维才是batch_size
        encoding = torch.cat((output[0], output[-1]), -1)#双向的lstm，就把两个都放进去
        return self.decoder(encoding)#把最后一个时间步的输出输入MLP


def get_parameter_number(model):#参数统计
    total_num = sum(p.numel() for p in model.parameters())
    trainable_num = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return {'Total': total_num, 'Trainable': trainable_num}



In [5]:
nn.init.orthogonal_(net.encoder.weight_ih_l0)

<ipython-input-5-a7d271f03ef5>:1: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  nn.init.orthogonal(net.encoder.weight_ih_l0)


Parameter containing:
tensor([[-0.0409, -0.0177,  0.0021,  ..., -0.0156, -0.0236,  0.0859],
        [-0.0106, -0.0140,  0.0257,  ...,  0.0113, -0.0064, -0.0088],
        [ 0.0393, -0.0069, -0.0356,  ...,  0.0377, -0.0182, -0.0116],
        ...,
        [ 0.0200, -0.0124, -0.0502,  ..., -0.0126,  0.1104,  0.0156],
        [ 0.0133,  0.0378,  0.0304,  ...,  0.0186, -0.0116, -0.0174],
        [ 0.0082, -0.0084,  0.0147,  ...,  0.0748, -0.0109,  0.0317]],
       device='cuda:0', dtype=torch.float64, requires_grad=True)

In [3]:
net = Lstm().double().cuda()

In [6]:
nn.init.orthogonal_(net.encoder.weight_ih_l0)
nn.init.orthogonal_(net.encoder.weight_hh_l0)
nn.init.constant_(net.encoder.bias_ih_l0,0.0)
nn.init.constant_(net.encoder.bias_hh_l0,0.0)
nn.init.normal_(net.decoder[0].weight,mean=0.0)
nn.init.constant_(net.decoder[0].bias, 0.0)
nn.init.normal_(net.decoder[3].weight,mean=0.0)
nn.init.constant_(net.decoder[3].bias, 0.0)

Parameter containing:
tensor([0., 0., 0.], device='cuda:0', dtype=torch.float64, requires_grad=True)

In [12]:
net.decoder[0].weight

Parameter containing:
tensor([[-0.0592, -0.2267, -1.2859,  ...,  0.3663,  1.2434,  0.7504],
        [-0.1745,  0.6647,  0.7540,  ...,  1.3438,  0.2506, -0.9482],
        [-0.3203, -0.4670,  1.3536,  ...,  1.3486,  0.5889, -0.4122],
        ...,
        [-1.4850,  0.1521, -0.2175,  ...,  0.9972,  1.6668, -0.5900],
        [-0.0294, -0.6558,  0.0816,  ...,  0.4105, -1.7296, -0.0425],
        [-0.8345,  0.0702, -0.8363,  ..., -0.3303,  0.5466, -1.2048]],
       device='cuda:0', dtype=torch.float64, requires_grad=True)

In [29]:
net.decoder[0].weight

Parameter containing:
tensor([[ 0.0195, -0.0097, -0.0268,  ..., -0.0120, -0.0077, -0.0137],
        [-0.0163,  0.0201,  0.0267,  ..., -0.0108, -0.0211, -0.0309],
        [-0.0255,  0.0022,  0.0272,  ..., -0.0213, -0.0178, -0.0257],
        ...,
        [-0.0099,  0.0077, -0.0252,  ..., -0.0087, -0.0274,  0.0179],
        [ 0.0125,  0.0228, -0.0296,  ...,  0.0153, -0.0261,  0.0166],
        [ 0.0096,  0.0283, -0.0115,  ..., -0.0198, -0.0155, -0.0059]],
       device='cuda:0', dtype=torch.float64, requires_grad=True)

In [12]:
lr, num_epochs = 0.001, 2000
optimizer= Nadam(net.parameters(), lr=lr)

In [14]:
checkpoint_path2 = '/home/jsy/log/checkpoints/checkpoints2.pth'

In [10]:
epoch =1

In [40]:
checkpoint2 = {
                'epoch': epoch,
                'model_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }
torch.save(checkpoint2, checkpoint_path2)

In [41]:
checkpoint2 = torch.load(checkpoint_path2)

In [23]:
help(checkpoint)

Help on dict object:

class dict(object)
 |  dict() -> new empty dictionary
 |  dict(mapping) -> new dictionary initialized from a mapping object's
 |      (key, value) pairs
 |  dict(iterable) -> new dictionary initialized as if via:
 |      d = {}
 |      for k, v in iterable:
 |          d[k] = v
 |  dict(**kwargs) -> new dictionary initialized with the name=value pairs
 |      in the keyword argument list.  For example:  dict(one=1, two=2)
 |  
 |  Built-in subclasses:
 |      StgDict
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, key, /)
 |      True if the dictionary has the specified key, else False.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>va

In [48]:
checkpoint2

{'epoch': 1,
 'model_state_dict': OrderedDict([('encoder.weight_ih_l0',
               tensor([[ 0.0552, -0.0449, -0.0156,  ...,  0.0248,  0.0038,  0.0071],
                       [-0.0522,  0.0124,  0.0338,  ...,  0.0412, -0.0465,  0.0118],
                       [ 0.0506, -0.0063,  0.0397,  ...,  0.0221, -0.0266, -0.0509],
                       ...,
                       [ 0.0570,  0.0541, -0.0444,  ..., -0.0081,  0.0184, -0.0403],
                       [-0.0088, -0.0408,  0.0528,  ...,  0.0513,  0.0032,  0.0169],
                       [-0.0312,  0.0469, -0.0059,  ...,  0.0368, -0.0515, -0.0048]],
                      device='cuda:0', dtype=torch.float64)),
              ('encoder.weight_hh_l0',
               tensor([[ 0.0299, -0.0295,  0.0107,  ..., -0.0063,  0.0462, -0.0452],
                       [ 0.0320,  0.0348, -0.0423,  ...,  0.0376, -0.0398,  0.0431],
                       [ 0.0420, -0.0071, -0.0231,  ..., -0.0275, -0.0229, -0.0282],
                       ...,
     

In [46]:
net.load_state_dict(checkpoint)

AttributeError: 'function' object has no attribute 'copy'

In [47]:
checkpoint

<bound method Module.state_dict of Lstm(
  (encoder): LSTM(10, 250, bidirectional=True)
  (decoder): Sequential(
    (0): Linear(in_features=1000, out_features=250, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=250, out_features=3, bias=True)
  )
)>

In [52]:
optimizer.__getstate__()

{'defaults': {'lr': 0.001,
  'betas': (0.9, 0.999),
  'eps': 1e-08,
  'weight_decay': 0,
  'schedule_decay': 0.004},
 'state': defaultdict(dict, {}),
 'param_groups': [{'params': [Parameter containing:
    tensor([[ 0.0552, -0.0449, -0.0156,  ...,  0.0248,  0.0038,  0.0071],
            [-0.0522,  0.0124,  0.0338,  ...,  0.0412, -0.0465,  0.0118],
            [ 0.0506, -0.0063,  0.0397,  ...,  0.0221, -0.0266, -0.0509],
            ...,
            [ 0.0570,  0.0541, -0.0444,  ..., -0.0081,  0.0184, -0.0403],
            [-0.0088, -0.0408,  0.0528,  ...,  0.0513,  0.0032,  0.0169],
            [-0.0312,  0.0469, -0.0059,  ...,  0.0368, -0.0515, -0.0048]],
           device='cuda:0', dtype=torch.float64, requires_grad=True),
    Parameter containing:
    tensor([[ 0.0299, -0.0295,  0.0107,  ..., -0.0063,  0.0462, -0.0452],
            [ 0.0320,  0.0348, -0.0423,  ...,  0.0376, -0.0398,  0.0431],
            [ 0.0420, -0.0071, -0.0231,  ..., -0.0275, -0.0229, -0.0282],
            ...,
 

In [53]:
help(optimizer)

Help on Nadam in module pywick.optimizers.nadam object:

class Nadam(torch.optim.optimizer.Optimizer)
 |  Nadam(params, lr=0.002, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, schedule_decay=0.004)
 |  
 |  Implements Nadam algorithm (a variant of Adam based on Nesterov momentum).
 |  
 |  It has been proposed in `Incorporating Nesterov Momentum into Adam`__.
 |  
 |  :param params: (iterable): iterable of parameters to optimize or dicts defining
 |      parameter groups
 |  :param lr: (float, optional): learning rate (default: 2e-3)
 |  :param betas: (Tuple[float, float], optional): coefficients used for computing
 |      running averages of gradient and its square
 |  :param eps: (float, optional): term added to the denominator to improve
 |      numerical stability (default: 1e-8)
 |  :param weight_decay: (float, optional): weight decay (L2 penalty) (default: 0)
 |  :param schedule_decay: (float, optional): momentum schedule decay (default: 4e-3)
 |  
 |  __ http://cs229.stanford.e

In [56]:
optimizer.args()

AttributeError: 'Nadam' object has no attribute 'args'